In [1]:
import os
import librosa
import argparse
import numpy as np
import pandas as pd
from pathlib import Path
from pesq import pesq
from pystoi import stoi
import mir_eval

In [2]:
def getSeparationMetrics(audio1, audio2, audio1_gt, audio2_gt):
    reference_sources = np.concatenate((np.expand_dims(audio1_gt, axis=0), np.expand_dims(audio2_gt, axis=0)), axis=0)
    estimated_sources = np.concatenate((np.expand_dims(audio1, axis=0), np.expand_dims(audio2, axis=0)), axis=0)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)


def process_folder(results_dir, audio_sampling_rate,mic_index=0):
    # Find the wav files in the directory

    wav_files = [f for f in os.listdir(results_dir) if f.endswith('.wav') and f'mic{mic_index}' in f]

    print(f'the results_dir is: {results_dir}')


    # Filter out mixed files
    separated_files = [f for f in wav_files if 'separated' in f]
    ground_truth_files = [f for f in wav_files if 'separated' not in f and 'mixed' not in f]

    if len(separated_files) != 2 or len(ground_truth_files) != 2:
        print(f"Skipping folder {results_dir} due to mismatched file counts.")
        return None
    
    #print(separated_files)
    #print(ground_truth_files)
    
    # Sort files to match separated with ground truth
    separated_files.sort()
    ground_truth_files.sort()

    # Assume naming convention: 001.wav -> 001_separated.wav
    audio1_gt_path = Path(os.path.join(results_dir, ground_truth_files[0]))
    audio2_gt_path = Path(os.path.join(results_dir, ground_truth_files[1]))
    audio1_path = Path(os.path.join(results_dir, separated_files[0]))
    audio2_path = Path(os.path.join(results_dir, separated_files[1]))

    print(f'the audio1_gt_path is: {audio1_gt_path}')
    print(f'the audio2_gt_path is: {audio2_gt_path}')
    print(f'the audio1_path is: {audio1_path}')
    print(f'the audio2_path is: {audio2_path}')

    audio1, _ = librosa.load(audio1_path, sr=audio_sampling_rate)
    audio2, _ = librosa.load(audio2_path, sr=audio_sampling_rate)
    audio1_gt, _ = librosa.load(audio1_gt_path, sr=audio_sampling_rate)
    audio2_gt, _ = librosa.load(audio2_gt_path, sr=audio_sampling_rate)

    sdr, sir, sar = getSeparationMetrics(audio1, audio2, audio1_gt, audio2_gt)
    print(f'sdr: {sdr}\n'
          f'sir: {sir}\n'
          f'sar: {sar}')

    stoi_score1 = stoi(audio1_gt, audio1, audio_sampling_rate, extended=False)
    stoi_score2 = stoi(audio2_gt, audio2, audio_sampling_rate, extended=False)
    stoi_score = (stoi_score1 + stoi_score2) / 2
    print(f'stoi_score: {stoi_score}')
    
    pesq_score1 = pesq(audio_sampling_rate, audio1, audio1_gt, 'wb')
    pesq_score2 = pesq(audio_sampling_rate, audio2, audio2_gt, 'wb')
    pesq_score = (pesq_score1 + pesq_score2) / 2
    print(f'pesq score is: {pesq_score}')

    return sdr, sir, sar, pesq_score, stoi_score


In [27]:
def main():
    test_dir = "C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex"
    audio_sampling_rate =16000
    mic_index = 3
   
    # DataFrame to store all results
    results_df = pd.DataFrame(columns=['Folder', 'SDR', 'SIR', 'SAR', 'PESQ', 'STOI'])
    test_dirs = os.listdir(test_dir)
    #test_dirs.remove('0000000__Evaluation')

    # Loop over folders in the test directory
    for folder_name in test_dirs:
        folder_path = os.path.join(test_dir, folder_name)
        print(folder_path)
        if os.path.isdir(folder_path):
            print('Processing folder:', folder_name)
            sdr, sir, sar, pesq_score, stoi_score = process_folder(folder_path, audio_sampling_rate, mic_index)
            results_df = pd.concat([results_df, pd.DataFrame([{
                'Folder': folder_name,
                'SDR': sdr,
                'SIR': sir,
                'SAR': sar,
                'PESQ': pesq_score,
                'STOI': stoi_score
            }])], ignore_index=True)

    # Calculate averages and add them to the DataFrame
    averages = results_df.mean(numeric_only=True)
    averages['Folder'] = 'Average'
    averages_df = pd.DataFrame([averages])
    results_df = pd.concat([results_df, averages_df], ignore_index=True)

    excel_path = os.path.join(test_dir, f'evaluation_results_microphone_{mic_index}.xlsx')
    try:
        results_df.to_excel(excel_path, index=False)
        print(f"Results saved to {excel_path}")
    except PermissionError as e:
        print(f"PermissionError: {e}. The file might be open in another program or you might not have write permissions.")
    except Exception as e:
        print(f"An error occurred while saving the results: {e}")

In [28]:
 if __name__ == '__main__':
    main()

C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex\evaluation_results_microphone_0.xlsx
C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex\evaluation_results_microphone_1.xlsx
C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex\evaluation_results_microphone_2.xlsx
C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex\id04030_7mXUMuo5_NE_00002VSid06913_Y60BPD7Ao1U_00089
Processing folder: id04030_7mXUMuo5_NE_00002VSid06913_Y60BPD7Ao1U_00089
the results_dir is: C:/Users/yosra/Documents/AV-speech-separation/data/test/results/simulated/complex\id04030_7mXUMuo5_NE_00002VSid06913_Y60BPD7Ao1U_00089
the audio1_gt_path is: C:\Users\yosra\Documents\AV-speech-separation\data\test\results\simulated\complex\id04030_7mXUMuo5_NE_00002VSid06913_Y60BPD7Ao1U_00089\00002_mic3_voice0.wav
the audio2_gt_path is: C:\Users\yosra\Documents\AV-speech-separation\data\test\results\simulated\comp